In [290]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from time import process_time
from sklearn.cluster import DBSCAN


In [2]:
#Load zones 20, 19, and 18
t0_start = process_time() 

AIS_raw_20 = pd.read_csv('AIS_2017_12_Zone20.csv', header=0)
AIS_raw_19 = pd.read_csv('AIS_2017_12_Zone19.csv', header=0)
AIS_raw_18 = pd.read_csv('AIS_2017_12_Zone18.csv', header=0)

t0_end = process_time()
load_time = t0_end-t0_start
loaded_rows = len(AIS_raw_20)+len(AIS_raw_19)+len(AIS_raw_18)

Load Time:  132.96875
Number of rows:  33959936


In [3]:
#Merge dataframes into a single dataframe
t1_start = process_time() 

zones_comb = pd.concat([AIS_raw_20, AIS_raw_19, AIS_raw_18]).reset_index(drop=True)

t1_end = process_time()
combine_time = t1_end-t1_start

Combine Time:  95.140625
Number of rows:  33959936


In [4]:
# Some metrics of the NaN values and number of ships
print("Percentage of NaN values: \n",zones_comb.isna().sum()/(zones_comb.count()+zones_comb.isna().sum()))
print("Number of ships in zone 18,19,20: ", len(zones_comb["MMSI"].unique()))

Percentage of NaN values: 
 MMSI            0.000000
BaseDateTime    0.000000
LAT             0.000000
LON             0.000000
SOG             0.000000
COG             0.000000
Heading         0.000000
VesselName      0.050550
IMO             0.421287
CallSign        0.113374
VesselType      0.085793
Status          0.199476
Length          0.134896
Width           0.241463
Draft           0.753950
Cargo           0.607525
dtype: float64
Number of ships in zone 18,19,20:  6618


In [94]:
# Clean data frame of the NaN values of status, length, width, and vessel type and only keep non-useful status values.

t2_start = process_time() 

zones_clean = zones_comb[['MMSI','BaseDateTime','LAT','LON','VesselType','Status','Length','Width']].dropna()
zones_clean = zones_clean[zones_clean['Status'].isin({'at anchor', 'moored', 'power-driven vessel pushing ahead or towing alongside', \
                                                    'power-driven vessel towing astern','under way using engine'})].reset_index(drop=True)
t2_end = process_time()
t2 = t2_end-t2_start

print("Clean Time: ", t2)
print("Number of rows: ", len(zones_clean))

Clean Time:  45.875
Number of rows:  19189015


In [95]:
# Further cleaning by dropping the ships that do not have a change in status

t3_start = process_time() 

zones_clean_grouped = zones_clean.groupby("MMSI")['Status'].apply(set).reset_index()

for index,row in zones_clean_grouped.iterrows():
    if len(row['Status']) < 2:
        zones_clean_grouped.drop(index, inplace=True) 
     
useful_ships = zones_clean_grouped['MMSI'].unique().tolist()
zones_useful = zones_clean[zones_clean['MMSI'].isin(useful_ships)]

t3_end = process_time()
t3 = t3_end-t3_start

print("Clean Time: ", t3)
print("Number of rows: ", len(zones_useful))

Clean Time:  18.671875
Number of rows:  7515590


In [7]:
zones_comb_list = zones_useful.groupby("MMSI").agg({'Status':list, 'LAT':list, 'LON':list, 'BaseDateTime':list, \
                                                    'VesselType':set, 'Length':set, 'Width':set}).reset_index()
zones_comb_list

,MMSI,Status,LAT,LON,BaseDateTime,VesselType,Length,Width
0,209008000,"[under way using engine, under way using engin...","[45.58832, 45.58039, 45.5711, 45.56266, 45.555...","[-73.50178000000001, -73.50298000000001, -73.5...","[2017-12-11T07:25:22, 2017-12-11T07:35:33, 201...",{1004.0},{184.93},{23.7}
1,209016000,"[under way using engine, under way using engin...","[45.5938, 45.5886, 45.58599, 45.57897, 45.5763...","[-73.5029, -73.50186, -73.50179, -73.50322, -7...","[2017-12-03T12:42:14, 2017-12-03T12:45:14, 201...",{1004.0},{185.01},{23.7}
2,209038000,"[under way using engine, under way using engin...","[36.54105, 36.54318, 36.557959999999994, 36.55...","[-73.70288000000001, -73.76671999999999, -73.9...","[2017-12-18T15:12:39, 2017-12-18T15:34:29, 201...",{1004.0},{255.24},{43.06}
3,209141000,"[at anchor, at anchor, at anchor, at anchor, a...","[18.02078, 18.02072, 18.0206, 18.02078, 18.020...","[-63.118469999999995, -63.118469999999995, -63...","[2017-12-04T19:06:12, 2017-12-04T20:18:13, 201...",{1004.0},{143.15},{22.8}
4,209263000,"[under way using engine, under way using engin...","[16.2268, 16.13701, 16.14542, 16.12731, 16.077...","[-65.97516999999999, -65.80495, -65.82134, -65...","[2017-12-05T02:20:18, 2017-12-05T03:16:26, 201...",{1004.0},{180.01},{30.0}
...,...,...,...,...,...,...,...,...
1341,636092756,"[moored, moored, moored, moored, moored, moore...","[40.55792, 40.55793, 40.558170000000004, 40.55...","[-74.23808000000001, -74.23808000000001, -74.2...","[2017-12-01T00:03:58, 2017-12-01T00:09:12, 201...",{1024.0},{180.01},{32.2}
1342,636092765,"[under way using engine, under way using engin...","[29.158479999999997, 33.92338, 34.07495, 34.09...","[-76.33901999999999, -77.99745, -77.93315, -77...","[2017-12-10T13:17:28, 2017-12-16T20:49:52, 201...",{74.0},{260.0},{32.0}
1343,671051000,"[under way using engine, under way using engin...","[18.082629999999998, 18.082629999999998, 18.08...","[-63.084619999999994, -63.08461, -63.084619999...","[2017-12-05T10:00:17, 2017-12-05T09:57:38, 201...",{1012.0},{63.89},{13.59}
1344,671932000,"[under way using engine, under way using engin...","[18.431810000000002, 18.433729999999997, 18.43...","[-65.37468, -65.39726999999999, -65.404, -65.3...","[2017-12-01T00:01:00, 2017-12-01T00:08:50, 201...",{1004.0},{51.26},{11.59}


In [122]:
# Group the data by MMSI (ships) then filter out values of lat and lon where this is not a change in status.

t4_start = process_time() 

zones_ordered = zones_useful.sort_values(['MMSI','BaseDateTime'],ascending=True)
zones_comb_list = zones_ordered.groupby("MMSI").agg({'Status':list, 'LAT':list, 'LON':list, 'BaseDateTime':list, \
                                                    'VesselType':set, 'Length':set, 'Width':set}).reset_index()

def compress_stat(row):
    stat = row['Status']
    time = row['BaseDateTime']
    lon = row['LON']
    lat = row['LAT']
    vals = [(stat[i+1],time[i+1],lon[i+1],lat[i+1]) for i,j,k in zip(np.arange(len(stat)-1), stat[:-1], stat[1:]) if (j!=k or i==0 )]
    return [(j[1],j[2],j[3]) for i,j in enumerate(vals) if j[0] in {'moored','at anchor'}]

zones_comb_list['Status_comp'] = zones_comb_list.apply(lambda row: compress_stat(row), axis=1)
zones_compressed = zones_comb_list.drop(['Status','LAT','LON','BaseDateTime'], axis=1)

for index,rows in zones_compressed.iterrows():  
    if len(rows['Status_comp']) < 2:
        zones_compressed.drop(index, inplace=True)

t4_end = process_time()
t4 = t4_end - t4_start

print("Combine Time: ", t4)
print("Number of total datapoints: ", sum([len(i) for i in zones_compressed_dict.values()]))
zones_compressed


Combine Time:  58.0
Number of total datapoints:  5848


,MMSI,VesselType,Length,Width,Status_comp
0,209008000,{1004.0},{184.93},{23.7},"[(2017-12-24T02:49:25, -75.04328000000001, 44...."
1,209016000,{1004.0},{185.01},{23.7},"[(2017-12-03T13:55:03, -73.53511999999999, 45...."
3,209141000,{1004.0},{143.15},{22.8},"[(2017-12-04T20:18:13, -63.118469999999995, 18..."
6,209532000,{1004.0},{180.0},{30.0},"[(2017-12-12T02:58:44, -76.57229, 39.267140000..."
7,209534000,{1004.0},{147.85},{23.25},"[(2017-12-07T21:28:24, -66.09277, 18.43661), (..."
...,...,...,...,...,...
1327,636092314,{1004.0},{134.65},{21.5},"[(2017-12-19T01:09:48, -73.53487, 45.540659999..."
1329,636092495,{1024.0},{116.9},{18.0},"[(2017-12-21T22:27:21, -65.61142, 18.08231), (..."
1333,636092684,{1024.0},{182.98},{32.23},"[(2017-12-21T06:21:16, -71.37097, 41.78075), (..."
1334,636092756,{1024.0},{180.01},{32.2},"[(2017-12-01T00:09:12, -74.23808000000001, 40...."


In [315]:

zones_expanded = zones_compressed.explode('Status_comp').reset_index(drop=True)
zones_expanded['Date-Time'] = [i[0] for i in zones_expanded['Status_comp'].to_list()]
zones_expanded['LON'] = [i[1] for i in zones_expanded['Status_comp'].to_list()]
zones_expanded['LAT'] = [i[2] for i in zones_expanded['Status_comp'].to_list()]
zones_expanded['Length'] = zones_expanded['Length'].apply(lambda row: list(row)[0])
zones_expanded['Width'] = zones_expanded['Width'].apply(lambda row: list(row)[0])
zones_expanded['VesselType'] = zones_expanded['VesselType'].apply(lambda row: list(row)[0])
zones_expanded = zones_expanded.drop('Status_comp', axis=1)

#Placeholder Carbon calculator
def carbonCalc(length,width,vessel):
    return length*width*vessel

zones_expanded['C02_output'] = zones_expanded.apply(lambda row: carbonCalc(row['Length'],row['Width'],row['VesselType']), axis=1)
zones_expanded


,MMSI,VesselType,Length,Width,Date-Time,LON,LAT,C02_output
0,209008000,1004.0,184.93,23.70,2017-12-24T02:49:25,-75.04328,44.94164,4.400372e+06
1,209008000,1004.0,184.93,23.70,2017-12-24T13:00:42,-74.78600,44.98705,4.400372e+06
2,209008000,1004.0,184.93,23.70,2017-12-24T19:41:19,-73.91337,45.32847,4.400372e+06
3,209016000,1004.0,185.01,23.70,2017-12-03T13:55:03,-73.53512,45.54063,4.402276e+06
4,209016000,1004.0,185.01,23.70,2017-12-08T05:03:47,-73.90338,45.33575,4.402276e+06
...,...,...,...,...,...,...,...,...
5843,671932000,1004.0,51.26,11.59,2017-12-08T09:03:17,-66.10304,18.46288,5.964798e+05
5844,671932000,1004.0,51.26,11.59,2017-12-18T01:57:03,-66.10866,18.45968,5.964798e+05
5845,671932000,1004.0,51.26,11.59,2017-12-19T21:26:50,-64.69856,17.74676,5.964798e+05
5846,671932000,1004.0,51.26,11.59,2017-12-21T00:00:03,-65.53047,18.44690,5.964798e+05


In [324]:

lon = zones_expanded['LON'].to_list()
lat = zones_expanded['LAT'].to_list()
Coords = np.array([[i,j] for i,j in zip(lon,lat)])
clustering = DBSCAN(eps=0.1, min_samples=2).fit(Coords)
#plt.scatter(lon,lat,c=clustering.labels_)

zones_expanded['Port_ID'] = clustering.labels_
port_reference = zones_expanded.groupby('Port_ID').mean()[['LON','LAT']]


In [325]:
zones_grouped = zones_expanded.groupby('MMSI').agg({'Port_ID':list}).reset_index()
len_list = zones_grouped['Port_ID'].apply(lambda x: len(x)).to_list()
len_list = np.cumsum(len_list)
len_list = [i-1 for i in len_list][:-1]
zones_graph = pd.DataFrame()
zones_graph['Start'] = zones_expanded['Port_ID'][:-1].to_list()
zones_graph['End'] = zones_expanded['Port_ID'][1:].to_list()
zones_graph['MMSI'] = zones_expanded['MMSI'][:-1].to_list()
zones_graph['Date-Time'] = zones_expanded['Date-Time'][:-1].to_list()
zones_graph['VesselType'] = zones_expanded['VesselType'][:-1].to_list()
zones_graph['ShipLength'] = zones_expanded['Length'][:-1].to_list()
zones_graph['ShipWidth'] = zones_expanded['Width'][:-1].to_list()
zones_graph['C02_output'] = zones_expanded['C02_output'][:-1].to_list()

#Kilometers (as the crow flies)
def distCalc(start,end):
    start_lat = math.radians(port_reference['LAT'][start])    
    end_lat = math.radians(port_reference['LAT'][end])    
    start_lon = math.radians(port_reference['LON'][start])    
    end_lon = math.radians(port_reference['LON'][end])    
    dlon = end_lon - start_lon
    dlat = end_lat - start_lat
    a = math.sin(dlat / 2)**2 + math.cos(start_lat) * math.cos(end_lat) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return 6373.0 * c

zones_graph = zones_graph.drop(len_list)
zones_graph['DistBetweenPorts'] = zones_graph.apply(lambda row: distCalc(row['Start'],row['End']), axis=1)
zones_graph.to_csv('Zones_graphed.csv', index=False)



In [327]:
zones_expanded.to_csv('Zones_expanded.csv', index=False)
port_reference.to_csv('Port_reference.csv')
zones_graph.to_csv('Zones_graphed.csv', index=False)